In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px # to plot the time series plot
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas import DataFrame
import os
import datetime
import IPython
import IPython.display

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import pandas as pd
from pandas import read_csv
import numpy as np
import random
from pandas import DataFrame
from numpy import concatenate
from pandas import concat
from scipy.stats import randint as sp_randint
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score
import matplotlib.patches as mpatches
import matplotlib.pylab as plt
import matplotlib as mpl
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 7)
plt.style.use('fast')
mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
from sklearn.neural_network import MLPRegressor
from tensorflow.keras import Sequential
from scikeras.wrappers import KerasClassifier, KerasRegressor
import keras_tuner
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D
from tensorflow.keras.layers import Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow as tf
from tensorflow import keras

In [16]:
# CARGAR LA BASE DE DATOS
from pandas import read_csv
datos = read_csv('Lebrija_Lags.csv', encoding='latin-1', sep=";")
# Dropping the id and date columns
datos = datos.drop(['Municipio', 'Año', 'Semana_Epi', 'Temp_Seca_Max_Dia_Prom_1','Temp_Seca_Max_Dia_Prom_2', 
                    'Temp_Seca_Max_Dia_Prom_3','Temp_Seca_Max_Dia_Prom_4', 'Temp_Seca_Max_Dia_Prom_5',
                    'Temp_Seca_Max_Dia_Prom_6', 'Temp_Seca_Max_Sem_1','Temp_Seca_Max_Sem_2', 'Temp_Seca_Max_Sem_3', 
                    'Temp_Seca_Max_Sem_4','Temp_Seca_Max_Sem_5', 'Temp_Seca_Max_Sem_6', 'Temp_Seca_Min_Dia_Prom_1', 
                    'Temp_Seca_Min_Dia_Prom_2', 'Temp_Seca_Min_Dia_Prom_3', 'Temp_Seca_Min_Dia_Prom_4',
                    'Temp_Seca_Min_Dia_Prom_5', 'Temp_Seca_Min_Dia_Prom_6', 'Temp_Seca_Min_Sem_1', 'Temp_Seca_Min_Sem_2',
                    'Temp_Seca_Min_Sem_3', 'Temp_Seca_Min_Sem_4', 'Temp_Seca_Min_Sem_5', 'Temp_Seca_Min_Sem_6', 
                    'Prom_Dia_Preci_1','Prom_Dia_Preci_2', 'Prom_Dia_Preci_3', 'Prom_Dia_Preci_4','Prom_Dia_Preci_5', 
                    'Prom_Dia_Preci_6','Total_Preci_Sem_1', 'Total_Preci_Sem_2', 'Total_Preci_Sem_3','Total_Preci_Sem_4', 
                    'Total_Preci_Sem_5', 'Total_Preci_Sem_6', 'Temp_Max_Dia_Prom_1', 'Temp_Max_Dia_Prom_2',
                    'Temp_Max_Dia_Prom_3', 'Temp_Max_Dia_Prom_4', 'Temp_Max_Dia_Prom_5','Temp_Max_Dia_Prom_6',
                    'Temp_Max_Sem_1','Temp_Max_Sem_2', 'Temp_Max_Sem_3', 'Temp_Max_Sem_4', 'Temp_Max_Sem_5',
                    'Temp_Max_Sem_6', 'Temp_Min_Dia_Prom_1','Temp_Min_Dia_Prom_2', 'Temp_Min_Dia_Prom_3', 
                    'Temp_Min_Dia_Prom_4','Temp_Min_Dia_Prom_5', 'Temp_Min_Dia_Prom_6',
                    'Temp_Min_Sem_1', 'Temp_Min_Sem_2', 'Temp_Min_Sem_3', 'Temp_Min_Sem_4',
                    'Temp_Min_Sem_5', 'Temp_Min_Sem_6'],axis=1)
datos.columns

Index(['Casos_Dengue', 'Temp_Seca_Max_Dia_Prom', 'Temp_Seca_Max_Sem',
       'Temp_Seca_Min_Dia_Prom', 'Temp_Seca_Min_Sem', 'Prom_Dia_Preci',
       'Total_Preci_Sem', 'Temp_Max_Dia_Prom', 'Temp_Max_Sem',
       'Temp_Min_Dia_Prom', 'Temp_Min_Sem'],
      dtype='object')

In [17]:
train_size = int(len(datos) * 0.7)
test_size = len(datos) - train_size
train, test = datos.iloc[0:train_size], datos.iloc[train_size:len(datos)]
print(len(train), len(test))

291 125


In [18]:
f_columns = ['Casos_Dengue', 'Temp_Seca_Max_Dia_Prom', 'Temp_Seca_Max_Sem', 'Temp_Seca_Min_Dia_Prom', 'Temp_Seca_Min_Sem',
             'Prom_Dia_Preci','Total_Preci_Sem', 'Temp_Max_Dia_Prom', 'Temp_Max_Sem', 'Temp_Min_Dia_Prom', 'Temp_Min_Sem']
f_transformer = MinMaxScaler(feature_range=(0, 1))
f_transformer = f_transformer.fit(train[f_columns].to_numpy())
train.loc[:, f_columns] = f_transformer.transform(train[f_columns].to_numpy())
test.loc[:, f_columns] = f_transformer.transform(test[f_columns].to_numpy())

In [19]:
dengue_transformer = MinMaxScaler(feature_range=(0, 1))
dengue_transformer = dengue_transformer.fit(train[['Casos_Dengue']])
train['Casos_Dengue'] = dengue_transformer.transform(train[['Casos_Dengue']])
test['Casos_Dengue'] = dengue_transformer.transform(test[['Casos_Dengue']])

In [20]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [21]:
time_steps = 10
# reshape to [samples, time_steps, n_features]
X_train, y_train = create_dataset(train, train.Casos_Dengue, time_steps)
X_test, y_test = create_dataset(test, test.Casos_Dengue, time_steps)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape )

(281, 10, 11) (281,) (115, 10, 11) (115,)


In [22]:
def cnn_model(dropout_rate=0.2, optimizer="adam", activation="relu", No_of_CONV_and_Maxpool_layers=3,
              No_of_Units_in_dense_layers=100, padding ='same', Conv1d_filters=60):

    
    #model sequential
    cnn_model=Sequential()
    
    #CAPA DE ENTRADA
    cnn_model.add(keras.layers.Conv1D(32, kernel_size=2, strides=1,
                                      activation=activation, input_shape=(X_train.shape[1], X_train.shape[2]),padding=padding))
    
    
    for i in range(No_of_CONV_and_Maxpool_layers):
        
        cnn_model.add(keras.layers.Conv1D(Conv1d_filters, kernel_size=2, strides=1, activation=activation, padding=padding))
        cnn_model.add(keras.layers.MaxPooling1D(pool_size=2, padding=padding))
        
    cnn_model.add(Dropout(dropout_rate))
    cnn_model.add(Flatten())
    
    
    
    cnn_model.add(keras.layers.Dense(units=No_of_Units_in_dense_layers, activation=activation))
    cnn_model.add(Dropout(dropout_rate)) 
                  
    cnn_model.add(Dense(1))
    
    
    #COMPILACIÓN DEL MODELO
    cnn_model.compile(optimizer=optimizer, loss = 'mse', metrics = ['mse'])
    
    

    return cnn_model

In [23]:
#HIPERPARÁMETROS

activation=['relu','tanh', 'sigmoid', 'linear']
dropout_rate=[0.1, 0.3, 0.5]
No_of_CONV_and_Maxpool_layers=[2,3,4]
Conv1d_filters=[60,40,80,120,220]
No_of_Units_in_dense_layers=[128,64,32,256]
optimizer = ['adam', 'sgd', 'rmsprop']
padding = ['valid', 'same', 'causal']

param_grid = dict(model__activation=activation,
                  model__dropout_rate=dropout_rate,
                  model__padding=padding,
                  model__No_of_CONV_and_Maxpool_layers=No_of_CONV_and_Maxpool_layers,
                  model__Conv1d_filters=Conv1d_filters,
                  model__No_of_Units_in_dense_layers=No_of_Units_in_dense_layers,
                  model__optimizer=optimizer,)

In [24]:
from sklearn.model_selection import KFold
from sklearn.metrics import make_scorer

model =  KerasRegressor(model=cnn_model, verbose=0)

rs = RandomizedSearchCV(estimator=model, param_distributions=param_grid, scoring='neg_mean_squared_error',
                        n_jobs=-1, cv=KFold(3), n_iter=20, verbose=0,random_state = 123, return_train_score = True)

In [25]:
fitted_model = rs.fit(X_train, y_train)

In [26]:
resultados = pd.DataFrame(fitted_model.cv_results_)
resultados.filter(regex = '(param.*|mean_t|std_t)')\
    .drop(columns = 'params')\
    .sort_values('mean_test_score', ascending = False)\
    .head(10)

,param_model__padding,param_model__optimizer,param_model__dropout_rate,param_model__activation,param_model__No_of_Units_in_dense_layers,param_model__No_of_CONV_and_Maxpool_layers,param_model__Conv1d_filters,mean_test_score,std_test_score,mean_train_score,std_train_score
1,same,rmsprop,0.5,linear,256,3,80,-0.049072,0.010395,-0.046037,0.004005
10,valid,rmsprop,0.3,linear,128,2,60,-0.051661,0.011258,-0.043237,0.003181
18,same,rmsprop,0.5,relu,128,4,80,-0.052176,0.013579,-0.049536,0.003516
9,same,rmsprop,0.5,relu,32,3,220,-0.058272,0.018824,-0.053842,0.003226
6,same,sgd,0.3,linear,64,4,220,-0.058333,0.016531,-0.054326,0.003220
19,same,sgd,0.3,sigmoid,128,2,40,-0.062935,0.018814,-0.060910,0.006955
4,same,rmsprop,0.1,tanh,128,2,220,-0.064120,0.006086,-0.049222,0.007523
17,valid,sgd,0.1,relu,64,2,60,-0.066166,0.017077,-0.067490,0.013338
3,same,adam,0.3,sigmoid,64,2,120,-0.112319,0.049270,-0.102132,0.021268
12,valid,sgd,0.3,tanh,32,2,120,-0.113020,0.075815,-0.093322,0.062189


In [27]:
print("Best: %f using %s" % (fitted_model.best_score_, fitted_model.best_params_))

Best: -0.049072 using {'model__padding': 'same', 'model__optimizer': 'rmsprop', 'model__dropout_rate': 0.5, 'model__activation': 'linear', 'model__No_of_Units_in_dense_layers': 256, 'model__No_of_CONV_and_Maxpool_layers': 3, 'model__Conv1d_filters': 80}
